# Snow Depth Applications

## Author(s)

Karina Zikan, Zach Fair

## Existing Notebooks

[Snow Depth Applications](https://icesat-2-2023.hackweek.io/tutorials/snow-depth/applications-tutorial-snow-depth.html)

## Learning Outcomes

* Gain experience in working with SlideRule to access and pre-process ICESat-2 
data
* Learn how to use projections and interpolation to compare ICESat-2 track data 
with gridded raster products
* Develop a general understanding of how to measure snow depths with LiDAR, 
and learn about opportunities and challenges when using ICEsat-2 along-track
products


## Background
### How do we measure snow depth with LiDAR?
LiDAR is a useful tool for collecting high resolution snow depth maps over large spatial areas.

Snow depth is measured from LiDAR by differencing a snow-free LiDAR map from a snow-covered LiDAR map of the same area of interest.

TODO: Insert Figure 6 from Deems et al. here

TODO: Insert Figure 7b from Deems et al

### Can we do this with ICESat-2?
Yes! By differencing snow-covered ICESat-2 transects from snow-free maps, we can calculate snow depth!

Performing the calculation with ICESat-2 is a little different from other LiDAR snow depth methods, given that ICESat-2 is a transect of points rather than gridded raster data. ICESat-2 also has sparse coverage in the mid-latitudes, so generating an effective snow-covered or snow-free map will be difficult.

Because of these limitations, we need an independently-collected snow-free map of a region of interest for comparison. We also need to process the snow-free data into a form that can be differenced from snow-on ICESat-2 data.

In this tutorial, we will show an example of how to compare ICESat-2 data to raster data.

TODO: Add Karina's image over Dry Creek

### What do we need to calculate snow depth from ICESat-2?
1. A region of interest, where snow-free (and snow-covered, for validation) digital elevation models (DEMs) are available.
2. ICESat-2 data, ideally from one of the lower-level products (ATL03, ATL06, ATL08, Sliderule Earth).
3. A snow-free reference DEM for the snow depth calculation.

### What do we need to consider when comparing ICESat-2 and raster data?
**Geolocation**: To obtain usable results, it is important that we properly align the snow-free raster data with ICESat-2. Even small offsets can create large errors that worsen in rugged terrain.

TODO: Add Figure 9 from Nuth and Kabb

**Vegetation**: Incorrectly categorized vegetation returns can positively bias ground or snow surface height estimation. Additionally, dense vegetation can reduce the number of photon returns, thereby increasing uncertainty in our height estimates.

**Slope Effects**: Rugged terrain increases uncertainty in ICESat-2 returns and increases the impact of geolocation offsets between ICESat-2 and raster data. Additionally, steep slopes can negatively bias ground or snow surface height estimates.

### Computing Environment
We'll be using the following open source Python libraries in this notebook:

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rioxarray as rxr
import matplotlib.pyplot as plt
from sliderule import icesat2
from scipy.interpolate import RectBivariateSpline

### Data
We will use SlideRule to acquire customized ATL06 data. Specific customizations that we will implement include footprint averaging (i.e., along-track sampling rate) and photon identification (signal/noise and ground/vegetation).

We will look at snow depth data over Upper Kuparuk/Toolik (UKT) on the Arctic North Slope of Alaska. Because UKT is a relatively flat region with little vegetation, we should expect good agreement between ICESat-2 and our rasters of interest.

### Initialize SlideRule

In [2]:
icesat2.init("slideruleearth.io")

### Define Region of Interest
After we initialize SlideRule, we define our region of interest. Notice that there are two options given below. This is because SlideRule accepts either the coordinates of a box/polygon or a geoJSON for its `region` input.

We are going to use the bounding box method in this tutorial, but the syntax for the geoJSON method is included for the user's reference.

In [ ]:
# Define region of interest over Toolik, Alaska
